## Surface water flow and erosion simulation

Here we compute surface runoff, and sediment transport using existing
path sampling model.


Resources:

    
* [
GRASS GIS overview and manual](https://grass.osgeo.org/grass76/manuals/index.html)
    
* [GRASS book](http://www.grassbook.org/)




For animating in GRASS GIS 7, see the
[video instructions](https://www.youtube.com/watch?v=Pi1ESk9iPwk) or
instructions for the
[
  Spatio-temporal data handling and visualization in GRASS GIS workshop
](http://ncsu-geoforall-lab.github.io/grass-temporal-workshop/)


### Start GRASS GIS
Start GRASS - click on GRASS icon or type

In [ ]:
# This is a quick introduction into Jupyter Notebook.
# Python code can be executed like this:
a = 6
b = 7
c = a * b
print "Answer is", c
# Python code can be mixed with command line code (Bash).
# It is enough just to prefix the command line with an exclamation mark:
!echo "Answer is $c"
# Use Shift+Enter to execute this cell. The result is below.

In [ ]:
import os
import sys
import subprocess
from IPython.display import Image

# create GRASS GIS runtime environment
gisbase = subprocess.check_output(["grass", "--config", "path"]).strip()
os.environ['GISBASE'] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# do GRASS GIS imports
import grass.script as gs
import grass.script.setup as gsetup

# set GRASS GIS session data
rcfile = gsetup.init(gisbase, "/home/jovyan/grassdata", "nc_spm_08_grass7", "user1")

In [ ]:
# default font displays
os.environ['GRASS_FONT'] = 'sans'
# overwrite existing maps
os.environ['GRASS_OVERWRITE'] = '1'
gs.set_raise_on_error(True)
gs.set_capture_stderr(True)

In [ ]:
# set display modules to render into a file (named map.png by default)
os.environ['GRASS_RENDER_IMMEDIATE'] = 'cairo'
os.environ['GRASS_RENDER_FILE_READ'] = 'TRUE'
os.environ['GRASS_LEGEND_FILE'] = 'legend.txt'

In startup pannel set GRASS GIS Database Directory to path to datasets,
for example on MS Windows, `C:\Users\myname\grassdata`.
For GRASS Location select nc_spm_08_grass7 (North Carolina, State Plane, meters) and
for GRASS Mapset create a new mapset (called e.g. HW_interpolation_2).
Click Start GRASS session.


Change working directory:
_Settings_ > _GRASS working environment_ > _Change working directory_ > select/create any directory
or type `cd` (stands for change directory) into the GUI
_Console_ and hit Enter:

Download all text files with color rules (see above)
to the selected directory. Now you can use the commands from the assignment requiring the text file
without the need to specify the full path to the file.

### Overland flow depth and discharge

In [ ]:
gs.parse_command('g.region', region="rural_1m", res="2", flags='pg')

Calculate partial derivatives to define the gradient of elevation surface:

In [ ]:
gs.run_command('v.surf.rst', input="elev_lid792_bepts", elevation="elev_lid792_2m", slope="dx_2m", aspect="dy_2m", tension="15", smooth="1.5", npmin="150", flags='d')

Note that partial derivatives can be also computed using _r.slope.aspect_.


Compute spatial pattern of overland flow depth and discharge
by running the SIMWE model implemented in
the _r.sim.water_ module.
We will use uniform rainfall excess, infiltration and land cover.

In [ ]:
gs.run_command('r.sim.water', elevation="elev_lid792_2m", dx="dx_2m", dy="dy_2m", rain_value="50", infil_value="0", man_value="0.05", depth="wdp_2m", discharge="disch_2m", nwalkers="100000", niterations="120", output_step="20", flags='t')

Display the results, add legend for individual raster and whatever you think is appropriate.

In [ ]:
gs.run_command('d.rast', map="wdp_2m.120")
gs.run_command('d.rast', map="disch_2m.120")
Image(filename="map.png")

Optionally, animate the time series:
_File > Animation tool_


#### Peak runoff with predefined flow direction

Compute peak runoff with predefined flow direction
along the stream inluding culvert under the road.


First, compute direction (aspect) of the given streams:

In [ ]:
gs.run_command('v.to.rast', input="streams", output="streams_dir_2m", use="dir")

Compute stream dx and dy using direction and slope equal to 2 degrees:

In [ ]:
gs.mapcalc("dxstr_2m = tan(2.)*cos(streams_dir_2m)")
gs.mapcalc("dystr_2m = tan(2.)*sin(streams_dir_2m)")

Compute combined DEM and stream dx and dy:

In [ ]:
gs.mapcalc("dxdemstr_2m = if(isnull(dxstr_2m), dx_2m, dxstr_2m)")
gs.mapcalc("dydemstr_2m = if(isnull(dystr_2m), dy_2m, dystr_2m)")

Run the model:

In [ ]:
gs.run_command('r.sim.water', elevation="elev_lid792_2m", dx="dxdemstr_2m", dy="dydemstr_2m", rain_value="50", infil_value="0", man_value="0.05", depth="wdpstr_2m", discharge="dischstr_2m", nwalkers="100000", niterations="120", output_step="20", flags='t')
gs.run_command('d.rast', map="dischstr_2m.120")
Image(filename="map.png")

Use Animation tool to animate the discharge.


#### Runoff for spatially variable landcover and rainfall excess

Compute runoff for spatially variable landcover
represented by spatially variable Mannings and rainfall excess.


We will use variable Mannings coefficient defined by reclassifying
land cover class. Here is the content of a file called
`land_to_mannings.txt` with reclassification rules for
the _r.recode_ module.

In [ ]:
%%file land_to_mannings.txt
1:1:0.9:0.9
2:2:0.5:0.5
3:3:0.01:0.01
4:4:0.03:0.03
5:5:0.01:0.01
6:6:0.05:0.05
7:7:0.1:0.1
8:8:0.1:0.1
9:9:0.9:0.9
10:10:0.03:0.03
11:11:0.5:0.5

In [ ]:
gs.run_command('r.recode', input="landcover_1m", output="mancover_2m", rules="land_to_mannings.txt")

Similarly, we will create raster with spatially variable rainfall excess
rates based on the land cover classes. We use file named
`land_to_rain.txt` to specify the rates for individual classes.

In [ ]:
%%file land_to_rain.txt
1:1:50:50
2:2:5:5
3:3:40:40
4:4:35:35
5:5:50:50
6:6:40:40
7:7:25:25
8:8:15:15
9:9:50.:50.
10:10:40:40
11:11:10:10

Again, we use the file as rules for the _r.recode_ module.

In [ ]:
gs.run_command('r.recode', input="landcover_1m", output="raincover_2m", rules="land_to_rain.txt")

Run the model:

In [ ]:
gs.run_command('r.sim.water', elevation="elev_lid792_2m", dx="dxdemstr_2m", dy="dydemstr_2m", rain="raincover_2m", infil_value="0", man="mancover_2m", depth="wdpstrcov_2m", discharge="distrcov_2m", nwalkers="100000", niterations="120", output_step="20", flags='t')

Display the static results.

In [ ]:
gs.run_command('d.rast', map="wdpstrcov_2m.120")
gs.run_command('d.legend', raster="wdpstrcov_2m.120")
gs.run_command('d.rast', map="distrcov_2m.120")
gs.run_command('d.legend', raster="distrcov_2m.120")
Image(filename="map.png")

Animate the time series using the Animation tool.


### Sediment flow rate, erosion and deposition

Compute sediment flow rate and net erosion/deposition using sediment
transport part of the SIMWE model implemented in the
the _r.sim.sediment_ module.
To make the computations faster, set region just to the upper part of
the watershed:

In [ ]:
gs.run_command('g.region', s="s+290")

Compute input transport capacity and detachment coefficient maps:

In [ ]:
gs.mapcalc("tranin = 0.001")
gs.mapcalc("detin = 0.001")

Compute input critical shear stress:

In [ ]:
gs.mapcalc("tauin = 0.01")

Run the model using the last depth from previous run:

In [ ]:
gs.run_command('g.copy', rast="wdp_2m.120,wdp_2m")
gs.run_command('r.sim.sediment', elevation="elev_lid792_2m", dx="dx_2m", dy="dy_2m", water_depth="wdp_2m", detachment_coeff="detin", transport_coeff="tranin", shear_stress="tauin", man_value="0.05", nwalkers="1000000", niterations="120", transport_capacity="tcapacity", tlimit_erosion_deposition="erdepmax", sediment_flux="sedflow", erosion_deposition="erdepsimwe")

Display these results after few seconds:

In [ ]:
gs.run_command('d.rast', map="tcapacity")
gs.run_command('d.rast', map="erdepmax")
Image(filename="map.png")

Display the final results:

In [ ]:
gs.run_command('d.rast', map="sedflow")
gs.run_command('d.legend', raster="sedflow")
gs.run_command('d.rast', map="erdepsimwe")
gs.run_command('d.legend', raster="erdepsimwe")
Image(filename="map.png")

In [ ]:
# end the GRASS session
os.remove(rcfile)